In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#retrieve all the postal code from website wikipedia

list_url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(list_url).text

In [3]:
soup = BeautifulSoup(source, 'xml')
table = soup.find('table')

In [7]:
#building the dataframe into columns

column_name = ['Postal code', 'Borough', 'Neighborhood']
df= pd.DataFrame(columns= column_name)

In [8]:
#retrieve all the borough, neighborhood and postal code

for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data

df.head()

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


As specified in the assignment, some boroughs are not assigned. Those have to be removed so we will only be taking care of the ones assigned.

In [9]:
#clean the useless data (not assigned)

df=df[df['Borough']!='Not assigned']

In [10]:
df[df['Neighborhood']=='Not assigned']=df['Borough']
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [11]:
df2=df.groupby('Postal code')['Neighborhood'].apply(lambda x: "%s" % ', '.join(x))
df2=df2.reset_index(drop=False)
df2.rename(columns={'Neighborhood':'Neighborhood_combined'},inplace=True)

In [12]:
df_merge = pd.merge(df, df2, on = 'Postal code')

In [13]:
df_merge.drop(['Neighborhood'], axis=1, inplace=True)

In [14]:
df_merge.drop_duplicates(inplace=True)

In [18]:
df_merge.rename(columns={'Neighborhood_combined':'Neighborhood'}, inplace=True)
df_merge.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park


In [20]:
df_merge.shape

(103, 3)

In [21]:
def get_geocode(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude,longitude

In [22]:
gloc_df=pd.read_csv('http://cocl.us/Geospatial_data')
gloc_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [23]:
gloc_df.rename(columns={'Postal Code':'Postal code'},inplace=True)
gloc_merge = pd.merge(gloc_df, df2, on='Postal code')

In [26]:
gloc_data=gloc_merge[['Postal code','Borough','Neighbourhood','Latitude','Longitude']]

KeyError: "['Neighbourhood', 'Borough'] not in index"